In [7]:
import pandas as pd

from src.utils import download_images 
#src\utils.py

In [8]:
import os 
os.getcwd()

'c:\\Users\\manas\\OneDrive\\Documents\\GitHub\\amazonml\\__pycache__'

In [9]:
import pandas as pd
df = pd.read_csv('src/sample_test.csv')

In [10]:
df_86 = df.sample(86)
df_86

,index,image_link,group_id,entity_name
52,52,https://m.media-amazon.com/images/I/51vwYpDz2t...,311997,width
39,39,https://m.media-amazon.com/images/I/51fAzxNm+c...,648011,width
63,63,https://m.media-amazon.com/images/I/61G8bvWOb-...,701880,item_weight
42,42,https://m.media-amazon.com/images/I/51jTe522S2...,564709,maximum_weight_recommendation
75,75,https://m.media-amazon.com/images/I/71afEPoRGs...,701880,maximum_weight_recommendation
...,...,...,...,...
76,76,https://m.media-amazon.com/images/I/71afEPoRGs...,701880,item_weight
29,29,https://m.media-amazon.com/images/I/51EBBqNOJ1...,483370,depth
46,46,https://m.media-amazon.com/images/I/51oaOP8qJl...,140266,width
49,49,https://m.media-amazon.com/images/I/51r7U52rh7...,860821,wattage


In [11]:
images_links = df_86['image_link'].to_list()

In [12]:
import cv2
import pytesseract
import numpy as np
import os
import matplotlib.pyplot as plt

# Function to display multiple images in a grid
def display_images(images, title):
    # Adjust the grid size for 86 images (e.g., 18 rows and 5 columns to accommodate)
    fig, axes = plt.subplots(18, 5, figsize=(15, 30))  # 18 rows, 5 columns (90 images capacity)
    fig.suptitle(title, fontsize=16)

    for i, img in enumerate(images):
        if i >= 86:  # Limit to 86 images
            break
        ax = axes[i // 5, i % 5]
        ax.imshow(img, cmap='gray')
        ax.axis('off')
        ax.set_title(f'Image {i+1}')
    
    plt.tight_layout()
    plt.show()

# Initialize an array to store the images
image_paths = [os.path.join('images', filename) for filename in os.listdir('images') if filename.endswith(('.png', '.jpg', '.jpeg'))]

# Read only up to 86 images
original_images = [cv2.imread(path) for path in image_paths[:86] if cv2.imread(path) is not None]



In [13]:
# Grayscale conversion
gray_images = [cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) for img in original_images]



In [14]:
# Apply Otsu's binarization
binary_images = [cv2.threshold(cv2.cvtColor(img, cv2.COLOR_BGR2GRAY), 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1] for img in original_images]

In [15]:
# 4. Noise removal using Median Blur and display
denoised_images = [cv2.medianBlur(binary, 3) for binary in binary_images]

In [16]:
# 5. Dilation and display
kernel = np.ones((1, 1), np.uint8)
dilated_images = [cv2.dilate(denoised, kernel, iterations=1) for denoised in denoised_images]


In [17]:
# 6. Erosion and display
eroded_images = [cv2.erode(dilated, kernel, iterations=1) for dilated in dilated_images]


In [18]:
# 7. Skew Correction (rotation) and display
def correct_skew(image):
    coords = np.column_stack(np.where(image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)
    return rotated
skew_corrected_images = [correct_skew(eroded) for eroded in eroded_images]

In [19]:
# 8. Resize images (increase size for better OCR performance)
resized_images = [cv2.resize(image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC) for image in skew_corrected_images]

In [20]:
# 9. Invert colors for white text on black background (optional, based on image content)
inverted_images = [cv2.bitwise_not(image) for image in resized_images]

In [21]:
# 10. OCR Processing on each preprocessed image
ocr_results = []
for img in original_images:
    text = pytesseract.image_to_string(img)
    ocr_results.append(text)

In [22]:
# Output OCR results for each image
for idx, text in enumerate(ocr_results):
    print(f"Text from image {idx+1}:\n{text}\n")

Text from image 1:
20CM

L5G



Text from image 2:
Deodorizing module Cat litter shovel

‘Adsorb harmful substances and reduce odor Regardless of soll sand,

‘wood sand, crystal sand,
filter unused cat litter



Text from image 3:
Blade Diameter 05am
Rated Voltage: nav
Frequency: soz
Rated input Power 2100w

foLoad Speed 300

0° HIM W330

Max Curing Capacity: 6 HD Wisoam

‘NsxCuting Capacity {9 HOAW20mn



Text from image 4:


Text from image 5:
sow=40W
470 Im



Text from image 6:
Ideal Bed & Armchair Cover

Neat size foldaway

Reversible; black on one
side & grey on the other

9010

29cm

Sturdy & lightweight

Q

ein 26 Ibs / 1.2 kg

a60em


Text from image 7:


Text from image 8:
“0.2”



Text from image 9:


Text from image 10:
7

44"

6.75 LBS



Text from image 11:
“—

Desktop Size

40"

<=

> 9 Ly XeU -,, QZ UI en


Text from image 12:
6.2 in/ 16cm

9.4 in / 24cm


Text from image 13:
Width

1C.M.

Length

9 Meter


Text from image 14:
69cm / 27.2inch



Text from image 15:
12V 

In [23]:
import re
import string

def preprocess_ocr_text(ocr_results):
  
    
    # Remove unwanted characters (e.g. punctuation, special characters)
    ocr_results = re.sub(r'[^a-zA-Z0-9\s]', '', ocr_results)

    # Convert to lowercase
    # ocr_results = ocr_results.lower()

    # Remove extra whitespace
    ocr_results = re.sub(r'\s+', ' ', ocr_results)

    # Remove leading and trailing whitespace
    ocr_results = ocr_results.strip()

    return ocr_results

In [24]:
prep_text = []
for text in ocr_results:
  preprocessed_text = preprocess_ocr_text(text)
  prep_text.append(preprocessed_text)


In [25]:
print(prep_text)

['20CM L5G', 'Deodorizing module Cat litter shovel Adsorb harmful substances and reduce odor Regardless of soll sand wood sand crystal sand filter unused cat litter', 'Blade Diameter 05am Rated Voltage nav Frequency soz Rated input Power 2100w foLoad Speed 300 0 HIM W330 Max Curing Capacity 6 HD Wisoam NsxCuting Capacity 9 HOAW20mn', '', 'sow40W 470 Im', 'Ideal Bed Armchair Cover Neat size foldaway Reversible black on one side grey on the other 9010 29cm Sturdy lightweight Q ein 26 Ibs 12 kg a60em', '', '02', '', '7 44 675 LBS', 'Desktop Size 40 9 Ly XeU QZ UI en', '62 in 16cm 94 in 24cm', 'Width 1CM Length 9 Meter', '69cm 272inch', '12V 42v Car Heating Powered aon fuse effectively prevent electrical overload universal for 412V power source cars', '', '12inches 305cm kzinches 6inches 95inches241cm 107cm', 'Size Comparison Example size Samsung S9', '', '61Inches 42 Inches E26 Medium Base', '', 'UG ZW98', 'GROWS WITH YOUR CHILD Accommodates children ages 36 Adjustable height lock', 'Tupi

In [26]:
for text in ocr_results:
    print(f" Text: {text}")

 Text: 20CM

L5G


 Text: Deodorizing module Cat litter shovel

‘Adsorb harmful substances and reduce odor Regardless of soll sand,

‘wood sand, crystal sand,
filter unused cat litter


 Text: Blade Diameter 05am
Rated Voltage: nav
Frequency: soz
Rated input Power 2100w

foLoad Speed 300

0° HIM W330

Max Curing Capacity: 6 HD Wisoam

‘NsxCuting Capacity {9 HOAW20mn


 Text: 
 Text: sow=40W
470 Im


 Text: Ideal Bed & Armchair Cover

Neat size foldaway

Reversible; black on one
side & grey on the other

9010

29cm

Sturdy & lightweight

Q

ein 26 Ibs / 1.2 kg

a60em

 Text: 
 Text: “0.2”


 Text: 
 Text: 7

44"

6.75 LBS


 Text: “—

Desktop Size

40"

<=

> 9 Ly XeU -,, QZ UI en

 Text: 6.2 in/ 16cm

9.4 in / 24cm

 Text: Width

1C.M.

Length

9 Meter

 Text: 69cm / 27.2inch


 Text: 12V 42v Car Heating  —
Powered aon ——__

fuse, effectively prevent electrical overload, universal for
412V power source cars


 Text: 
 Text: 12inches
/30.5cm

kzinches 6inches

9.5inches/24.1cm /10.7cm



In [27]:
import spacy
nlp = spacy.load('en_core_web_sm')

In [28]:
i=0;
for text in ocr_results:
    doc = nlp(text)
    for entity in doc.ents:
        if (entity.label_=="QUANTITY"):
            print(f" Entity: {entity.start},{entity.text}, Label: {entity.label_}")
         

        
    

 Entity: 25,29cm, Label: QUANTITY
 Entity: 38,1.2 kg, Label: QUANTITY
 Entity: 8,24cm, Label: QUANTITY
 Entity: 0,69cm / 27.2inch, Label: QUANTITY
 Entity: 8,9.5inches/24.1cm, Label: QUANTITY
 Entity: 2,4.2 Inches, Label: QUANTITY
 Entity: 60,86 cubic feet, Label: QUANTITY
 Entity: 11,0.6 Cubic Feet, Label: QUANTITY
 Entity: 8,68.3mm, Label: QUANTITY
 Entity: 39,27.8mm, Label: QUANTITY
 Entity: 48,42.0mm, Label: QUANTITY
 Entity: 30,3.8cm / 1.5inch, Label: QUANTITY
 Entity: 52,16.9in/43cm, Label: QUANTITY
 Entity: 62,19.7in/50cm, Label: QUANTITY
 Entity: 67,5.3in/13.5cm, Label: QUANTITY


In [29]:
import csv

# Open a CSV file to write the entity results
with open('ocr_entities.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    
    # Write the header for the CSV file
    writer.writerow(['id','Entity', 'Label'])
    
    # Loop through OCR results and extract entities
    for text in ocr_results:
        doc = nlp(text)
        for entity in doc.ents:
            if entity.label_ == "QUANTITY":
                # Write entity and label to the CSV
                writer.writerow([entity.start,entity.text, entity.label_])

print("CSV file 'ocr_entities.csv' has been created.")


CSV file 'ocr_entities.csv' has been created.


In [30]:
df = pd.read_csv('ocr_entities.csv')

In [31]:
df.head()


,id,Entity,Label
0,25,29cm,QUANTITY
1,38,1.2 kg,QUANTITY
2,8,24cm,QUANTITY
3,0,69cm / 27.2inch,QUANTITY
4,8,9.5inches/24.1cm,QUANTITY


In [32]:
for i in range(len(df)):
    print(df.iloc[i]['Entity'])

29cm
1.2 kg
24cm
69cm / 27.2inch
9.5inches/24.1cm
4.2 Inches
86 cubic feet
0.6 Cubic Feet
68.3mm
27.8mm
42.0mm
3.8cm / 1.5inch
16.9in/43cm
19.7in/50cm
5.3in/13.5cm


In [33]:
import csv
import pandas as pd
import re

# Mapping of short forms to full forms
unit_mapping = {
    'cm': 'centimetre',
    'in': 'inch',
    'n': 'inch',  # assuming 'n' is a typo and you meant 'in'
    'Cubic Feet': 'Cubic Feet',
    'mm': 'millimetre',
    'ft': 'foot',
    'inches': 'inches'
}

def convert_units(row):
    ud,value, label = row
    for short_form, full_form in unit_mapping.items():
        pattern = r'(\d+)(?:' + re.escape(' ') + re.escape(short_form) + '|' + re.escape(short_form) + ')'
        value = re.sub(pattern, r'\1 ' + full_form, value)
    return [ud,value, label]

with open('ocr_entities.csv', 'r') as input_file, open('output.csv', 'w', newline='') as output_file:
    reader = csv.reader(input_file)
    writer = csv.writer(output_file)
   
    for row in reader:
        # print(row)
        writer.writerow(convert_units(row))




# out=pd.read_csv('ocr_entities.csv')
# print(out) #for output


# df = pd.read_csv('output.csv')
# for i in range(len(df)):
#     print(df.iloc[i]['Entity'])

In [34]:
df = pd.read_csv('output_cleaned.csv')

In [35]:
print("ID", df.columns)

ID Index(['index', 'prediction'], dtype='object')


In [36]:
df = df.rename(columns={'id': 'index'})

In [37]:
df.to_csv('output_cleaned.csv', index=False)

In [38]:
print("Updated headers:", df.columns)

Updated headers: Index(['index', 'prediction'], dtype='object')


In [39]:
df = df.rename(columns={'Entity': 'prediction'})

In [40]:
df.to_csv('output_cleaned.csv', index=False)

In [41]:
print("Updated headers:", df.columns)

Updated headers: Index(['index', 'prediction'], dtype='object')


In [42]:
df = df.drop(columns=['Label'])

KeyError: "['Label'] not found in axis"

In [43]:
df.to_csv('output.csv', index=False)

In [44]:
print("Column removed successfully. Updated columns:", df.columns)

Column removed successfully. Updated columns: Index(['index', 'prediction'], dtype='object')


In [45]:
df.reset_index(drop=True, inplace=True)

In [77]:
# Check if any rows are missing (compare expected indices)
expected_indices = set(range(88))  # Assuming 0-87 is the range you're expecting
actual_indices = set(df.index)

missing_indices = expected_indices - actual_indices
extra_indices = actual_indices - expected_indices

In [78]:
# Print any issues with indices
print("Missing index in file:", missing_indices)
print("Extra index in file:", extra_indices)

Missing index in file: set()
Extra index in file: set()


In [79]:
df.to_csv('output_cleaned.csv', index=False)

In [85]:
result_df = pd.DataFrame(columns=['index', 'prediction'])

In [83]:
df

""
0
1
2
3
4
...
83
84
85
86


In [49]:
print("File cleaned and saved as 'output_cleaned.csv'")

File cleaned and saved as 'output_cleaned.csv'


In [73]:
import csv
import pandas as pd

# Read the CSV file into a pandas DataFrame
df = pd.read_csv('output.csv')

# Get the value of n from the number of rows in the DataFrame
n = len(df)

# Create an empty dictionary to store the data
m = {}

# Iterate over the rows of the DataFrame
for index, row in df.iterrows():
    if pd.notna(row['index']):
        a = int(row['index'])
        b = row['prediction']
        if a not in m:
            m[a] = []
        m[a].append(b)

# Create a new DataFrame to store the results
result_df = pd.DataFrame(columns=['index', 'prediction'])



In [74]:
result_df

,index,prediction


In [69]:
result_df.to_csv('output1.csv', index=False)